# Amazon SageMaker FeatureStore - Example with Vectors

Kernel `Python 3 (Data Science)` works well with this notebook.

In this notebook, you will learn how to create a feature group for a dummy created large dataset in the SageMaker Feature Store with vectors Base64 encoded. You will then learn how to ingest the feature columns into the created feature group (both the Online and the Offline store) using SageMaker Python SDK. You will also see how to get an ingested feature record from the Online store and perform an SQL query to fetch from Offline feature store using Athena API.

## Setup SageMaker FeatureStore

Let's start by setting up the SageMaker Python SDK and boto client.

In [ ]:
import sys

!{sys.executable} -m pip install sagemaker pandas numpy --upgrade --quiet

In [ ]:
import boto3
import json
import sagemaker
from sagemaker.session import Session

region = boto3.Session().region_name
s3_client = boto3.client('s3', region_name=region)

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

#### S3 Bucket Setup For The OfflineStore

SageMaker FeatureStore writes the data in the OfflineStore of a FeatureGroup to a S3 bucket owned by you. To be able to write to your S3 bucket, SageMaker FeatureStore assumes an IAM role which has access to it. The role is also owned by you.
Note that the same bucket can be re-used across FeatureGroups. Data in the bucket is partitioned by FeatureGroup.

Set the default s3 bucket name and it will be referenced throughout the notebook.

In [ ]:
# You can modify the following to use a bucket of your choosing
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = 'sagemaker-featurestore-large-vecors-demo'

print(default_s3_bucket_name)

Set up the IAM role. This role gives SageMaker FeatureStore access to your S3 bucket. 

<div class="alert alert-block alert-warning">
<b>Note:</b> In this example we use the default SageMaker role, assuming it has both <b>AmazonSageMakerFullAccess</b> and <b>AmazonSageMakerFeatureStoreAccess</b> managed policies. If not, please make sure to attach them to the role before proceeding.
</div>

In [ ]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print (role)

## Inspect Dataset

We create a large dataset with `list_size` size `dataframe`, with vectors `vector_size` size. 

In [ ]:
list_size = 10000
vector_size = 128

We create a random embeddings with 128 chars length and then create a dummy dataset 

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import io

embeddings = []
for _ in range(list_size):
    embeddings.append(np.random.rand(vector_size))

In [ ]:
transaction_id_list = [i+1 for i in range(list_size)]

In [ ]:
import random

transaction_amt_list = [random.randrange(1000, 10000) for i in range(list_size)]

In [ ]:
cards_list = [random.randrange(1, 4) for i in range(list_size)]

for i in range(list_size):
    if cards_list[i]==1:
        cards_list[i]="mastercard"
    if cards_list[i]==2:
        cards_list[i]="visa"
    if cards_list[i]==3:
        cards_list[i]="diners"

In [ ]:
d = {
    'TransactionID': transaction_id_list, 
    'TransactionAmt': transaction_amt_list, 
    'card': cards_list,
    'embeddings': embeddings
}
my_sample_data = pd.DataFrame(data=d)
my_sample_data

## Ingest Data into FeatureStore

In this step we will create the FeatureGroups representing the transaction and identity tables.

#### Define FeatureGroups

In [ ]:
from time import gmtime, strftime, sleep

my_sample_feature_group_name = 'my-sample-large-vectors-b64-feature-group-'+str(list_size) + '-'+str(vector_size) + '-'+strftime('%d-%H-%M-%S', gmtime())
my_sample_feature_group_name

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup

my_sample_feature_group = FeatureGroup(name=my_sample_feature_group_name, sagemaker_session=feature_store_session)

### Convert Vector Features to Base64

In order to save space and improve latency, we encode features which are vectors to Base64.

In [ ]:
from feature_store_utils import encode_vector_features

encode_vector_features(my_sample_data, ['embeddings'])
my_sample_data

### Convert Object to String

We will now cast all object dtypes to string. The SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.

In [ ]:
my_sample_data.info()

In [ ]:
from feature_store_utils import cast_object_to_string
import time

current_time_sec = int(round(time.time()))

cast_object_to_string(my_sample_data)

### Add `EventTime` needed

In [ ]:
# record identifier and event time feature names
record_identifier_feature_name = "TransactionID"
event_time_feature_name = "EventTime"

# append EventTime feature
my_sample_data[event_time_feature_name] = pd.Series([current_time_sec]*len(my_sample_data), dtype="float64")

# load feature definitions to the feature group. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.
my_sample_feature_group.load_feature_definitions(data_frame=my_sample_data); # output is suppressed

In [ ]:
my_sample_data

In [ ]:
my_sample_data.info()

#### Create a Feature Group in SageMaker Feature Store

In [ ]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")

my_sample_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True
)

wait_for_feature_group_creation_complete(feature_group=my_sample_feature_group)

Confirm the FeatureGroup has been created by using the DescribeFeatureGroup and ListFeatureGroups APIs.

In [ ]:
feature_group_describe_response = my_sample_feature_group.describe()
feature_group_describe_response

#### PutRecords into Feature Group

After the Feature Group has been created, we can put data into the Feature Group by using the PutRecord API. This API can handle high TPS and is designed to be called by different streams. The data from all of these Put requests is buffered and written to S3 in chunks. The files will be written to the offline store within a few minutes of ingestion.

In [ ]:
%%time 

my_sample_feature_group.ingest(
    data_frame=my_sample_data, max_workers=5, wait=True
)

To confirm that data has been ingested, we can quickly retrieve a record from the online store.

In [ ]:
record_identifier_value = str(1)

featurestore_runtime.get_record(FeatureGroupName=my_sample_feature_group_name, RecordIdentifierValueAsString=record_identifier_value)

#### Retrieve a record from the online store into a Pandas DataFrame 

We'll instantiate `FeatureStoreUtils` class and use the `get_record_to_df` function in the `feature_store_utils` helper file to retrieve a record from the online store into a Pandas DataFrame.

In [ ]:
from feature_store_utils import FeatureStoreUtils
my_sample_feature_store_utils = FeatureStoreUtils(FeatureGroupName=my_sample_feature_group_name)

In [ ]:
record_identifier_value = 3

df = my_sample_feature_store_utils.get_record_to_df(RecordIdentifierValueAsString=record_identifier_value)
df

Now, let's decode the features which are vectors back from Base64

In [ ]:
from feature_store_utils import decode_vector_features

decode_vector_features(df, ['embeddings'])
df

## Batch fetch multiple records from the Online Feature Store

Fetch a list of selected items from the feature group.
##### Up to 100 records can be fetched from an online Feature Store in a single batch operation.

In [ ]:
identifiers = [
    {
        'FeatureGroupName': my_sample_feature_group_name,
        'RecordIdentifiersValueAsString': ["1", "2","3"]
    }
]
        
batch_get_record_response = featurestore_runtime.batch_get_record(Identifiers=identifiers)
records = batch_get_record_response['Records']
records

#### Fetch a list of selected items from a single feature group into a Pandas DataFrame

We'll use the `batch_get_records_to_df` function in the `feature_store_utils` helper file to fetch a list of selected items from a single feature group into a Pandas DataFrame, and then use `decode_vector_features` to decode the features which are vectors back from Base64.

In [ ]:
record_identifiers_list = ["1", "2","3"]

batch_df = my_sample_feature_store_utils.batch_get_records_to_df(RecordIdentifiersValueAsString=record_identifiers_list)
decode_vector_features(batch_df, ['embeddings'])
batch_df

## Update a record in the Feature Store

We will update the record with `TransactionID` value of 1 with another `TransactionAmt` and update the `EventTime`.

Let's quickly retrieve this record from the online store.

In [ ]:
record_identifier_value = 1
my_sample_record = my_sample_feature_store_utils.get_record_to_df(RecordIdentifierValueAsString=record_identifier_value)
my_sample_record

In [ ]:
updated_time_sec = int(round(time.time()))

In [ ]:
my_sample_record["EventTime"] = updated_time_sec
my_sample_record["EventTime"] = my_sample_record["EventTime"].astype("float64")
my_sample_record["TransactionAmt"] = 99999

In [ ]:
my_sample_record

Now ingest the updated record into feature store.

In [ ]:
%%time 

my_sample_feature_group.ingest(
    data_frame=my_sample_record, max_workers=1, wait=True
)

To confirm that the record has been ingested and the data was updated, we can quickly retrieve this record from the online store.

In [ ]:
record_identifier_value = 1
my_sample_feature_store_utils.get_record_to_df(RecordIdentifierValueAsString=record_identifier_value)

## Delete a record from the Feature Store

We are now going to delete this record from the FeatureGroup.

In [ ]:
deleted_time_sec = str(round(time.time()))

In [ ]:
delete_record_response = featurestore_runtime.delete_record(
    FeatureGroupName=my_sample_feature_group_name,
    RecordIdentifierValueAsString=str(record_identifier_value),
    EventTime=deleted_time_sec
)
delete_record_response

To confirm that the record was deleted, we can quickly try to retrieve this record from the online store.

In [ ]:
record_identifier_value = 1
my_sample_feature_store_utils.get_record_to_df(RecordIdentifierValueAsString=record_identifier_value)

The SageMaker Python SDK’s FeatureStore class also provides the functionality to generate Hive DDL commands. Schema of the table is generated based on the feature definitions. Columns are named after feature name and data-type are inferred based on feature type.

In [ ]:
print(my_sample_feature_group.as_hive_ddl())

Now let's wait for the data to appear in our offline store before moving forward to creating a dataset. This will take approximately 5 minutes.

In [ ]:
s3_offline_location=feature_group_describe_response['OfflineStoreConfig']['S3StorageConfig']['ResolvedOutputS3Uri']
print(s3_offline_location)

my_sample_feature_group_s3_prefix = '/'.join(s3_offline_location.split("/")[3:])
print(my_sample_feature_group_s3_prefix)

offline_store_contents = None
while (offline_store_contents is None):
    objects_in_bucket = s3_client.list_objects(Bucket=default_s3_bucket_name,Prefix=my_sample_feature_group_s3_prefix)
    if ('Contents' in objects_in_bucket and len(objects_in_bucket['Contents']) > 1):
        offline_store_contents = objects_in_bucket['Contents']
    else:
        print('Waiting for data in offline store...\n')
        sleep(60)
    
print('Data available.')

## Build Training Dataset

SageMaker FeatureStore automatically builds the Glue Data Catalog for FeatureGroups (you can optionally turn it on/off while creating the FeatureGroup). In this example, we want to create one training dataset with FeatureValues from both identity and transaction FeatureGroups. This is done by utilizing the auto-built Catalog. We run an Athena query that fetch the data stored in the offline store in S3 for the FeatureGroup. 

Note the `eventtime` and `is_deleted` columns for the records with `TransactionID` value of 1 that were fetched. A new record is showing up now with a value of `True` in the `is_deleted` column.

In [ ]:
my_sample_query = my_sample_feature_group.athena_query()

my_sample_table = my_sample_query.table_name

query_string = 'SELECT * FROM "'+my_sample_table+'" order by transactionid'
print('Running ' + query_string)

# run Athena query. The output is loaded to a Pandas dataframe.
#dataset = pd.DataFrame()
my_sample_query.run(query_string=query_string, output_location='s3://'+default_s3_bucket_name+'/'+prefix+'/query_results/')
my_sample_query.wait()
dataset = my_sample_query.as_dataframe()

decode_vector_features(dataset, ['embeddings'])
dataset

## Cleanup Resources

Before deleting the Feature Group, let's inspect data size in the Offline Feature Group.

In [ ]:
fs_s3_uri = feature_group_describe_response['OfflineStoreConfig']['S3StorageConfig']['ResolvedOutputS3Uri']
print(fs_s3_uri)

In [ ]:
!aws s3 ls {fs_s3_uri}/ --recursive

In [ ]:
!aws s3 ls ${fs_s3_uri}/ --recursive | awk '{total += $3} END {print "Total:", total/1048576, "MB"}'

We can see that the Offline Feature Group data is around 177 MB. Le't delete it before we delete the feature group.

In [ ]:
!aws s3 rm {fs_s3_uri}/ --recursive

Now let's delete the Feature Group

In [ ]:
my_sample_feature_group.delete()